In [1]:
from pynytimes import NYTAPI
import datetime
import pandas as pd

import numpy as np

In [2]:
api_key = "uLZTQY03vZfRZfk6nmzKcSc82EEUj4lK"

In [4]:
def get_news(year, month, day, nyt=None, k=10):
    """
    get top k most relevent finance news headings on each day from NY times
    """
    res = []
    try:
        if not nyt:
            nyt = NYTAPI(api_key, parse_dates=True)

        articles = nyt.article_search(
                results = k,
                dates = {
                    "begin": datetime.datetime(year, month, day),
                    "end": datetime.datetime(year, month, day)
                },
                options = {
                    "sort": "relevance",
                    "news_desk": [
                        "Business", "Business Day", "Entrepreneurs", "Financial", "Technology"
                    ],
                    "section_name" : [
                        "Business", "Business Day", "Technology"
                    ]
                }
            )
        print('fetch done')
        for i in range(len(articles)):
            timestamp = articles[i]['pub_date']
            article_url = articles[i]['web_url']
            lead_paragraph = articles[i]['lead_paragraph']
            abstract = articles[i]['abstract']
            res.append({'timestamp': timestamp,
                        'article_url': article_url,
                        'lead_paragraph': lead_paragraph,
                        'abstract': abstract,
                        })
        
    except Exception as e:
        res.append({})
    
    return res


def generate_news(dates):
    """_summary_

    Parameters
    ----------
    dates : pd.Series 
        Series of Datetime objects.
    """
    nyt = NYTAPI(api_key, parse_dates=True)
    data = []
    for i, date in enumerate(dates):
        print(i)
        y, m, d = date.year, date.month, date.day
        news_list = get_news(y, m, d, nyt=nyt)
        data.extend(news_list)
            
    df = pd.DataFrame(data=data)
    return df

In [5]:
nyt = NYTAPI(api_key, parse_dates=True)


#### Get dates from our SPY data.

In [6]:
spy_df = pd.read_csv('../data/spy_prices.csv')
dates = pd.to_datetime(spy_df['Date'], utc=True)

len(dates)

1258

In [7]:
x = 10
res_df = generate_news(dates[:x])

0


KeyboardInterrupt: 

In [6]:
res_df.head()

NameError: name 'res_df' is not defined

In [105]:
threshold_time = pd.Timestamp('16:00:00').time()

res_df['adjusted_date'] = res_df['timestamp'].apply(lambda x: x.date() if x.time() < threshold_time else (x + pd.Timedelta(days=1)).date())
res_df.head()

,timestamp,article_url,lead_paragraph,abstract,adjusted_date
0,2024-09-23 23:07:00+00:00,https://www.nytimes.com/2024/09/23/business/de...,The Justice Department is preparing to sue Vis...,The agency plans to argue that the company ill...,2024-09-24
1,2024-09-23 19:54:50+00:00,https://www.nytimes.com/2024/09/23/business/ec...,Boeing on Monday made what it described as its...,The proposal includes raises of 30 percent ove...,2024-09-24
2,2024-09-23 15:11:02+00:00,https://www.nytimes.com/2024/09/23/technology/...,"On a recent afternoon, Tudor Achim gave a brai...",Chatbots like ChatGPT get stuff wrong. But res...,2024-09-23
3,2024-09-23 15:10:59+00:00,https://www.nytimes.com/2024/09/23/technology/...,As Jim Covello’s car barreled up Highway 101 f...,"Jim Covello, Goldman Sachs’s head of stock res...",2024-09-23
4,2024-09-23 12:03:18+00:00,https://www.nytimes.com/2024/09/23/business/de...,First came news that Qualcomm had made an info...,Rivals and at least one big investor are circl...,2024-09-23


In [106]:
res_df.to_csv("../data/nyt_headlines.csv", index=False)